In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

from snowflake.cortex import Complete
from snowflake.cortex import Summarize


In [ ]:
import streamlit as st

st.title("Hello Streamlit!")
st.write("This is your first interactive web app.")

In [ ]:

-- Welcome to Snowflake Notebooks!
-- Try out a SQL cell to generate some data.
SELECT 'FRIDAY' as SNOWDAY, 0.2 as CHANCE_OF_SNOW
UNION ALL
SELECT 'SATURDAY',0.5
UNION ALL 
SELECT 'SUNDAY', 0.9;

In [ ]:
# Then, we can use the python name to turn cell2 into a Pandas dataframe
my_df = cell2.to_pandas()

# Chart the data
st.subheader("Chance of SNOW ❄️")
st.line_chart(my_df, x='SNOWDAY', y='CHANCE_OF_SNOW')

# Give it a go!
st.subheader("Try it out yourself and show off your skills 🥇")

In [ ]:
trans = f"SELECT * FROM CALL_TRANSCRIPTS"
call_df = session.sql(trans).to_pandas()

In [ ]:
call_df.head()

In [ ]:
# --- Agent filter ---
agents = call_df['AGENT_NAME'].unique()
selected_agent = st.selectbox("Select Agent", agents)
filtered_df = call_df[call_df['AGENT_NAME'] == selected_agent]

In [ ]:
# --- Display data ---
st.subheader(f"Call Transcripts for Agent: {selected_agent}")
st.dataframe(filtered_df[['CALL_DATE', 'CUSTOMER_NAME', 'TRANSCRIPT']])

In [ ]:
# --- Display data ---
st.subheader(f"Call Transcripts for Agent: {selected_agent}")
st.dataframe(filtered_df[['CALL_DATE', 'CUSTOMER_NAME', 'TRANSCRIPT']])

In [ ]:
# --- Select call ---
selected_call = st.selectbox("Select a call to analyze", filtered_df['CALL_ID'])
selected_transcript = filtered_df[filtered_df['CALL_ID'] == selected_call]['TRANSCRIPT'].values[0]

In [ ]:
selected_transcript

In [ ]:
## Assume selected_transcript is already defined in Python
query = f"""
    SELECT 
        SNOWFLAKE.CORTEX.SENTIMENT($${selected_transcript}$$) AS SENTIMENT
"""
senti_result = session.sql(query).collect()
activ_senti_res = senti_result[0]['SENTIMENT']
print(activ_senti_res)

In [ ]:
summary_query = f"""
    SELECT 
        SNOWFLAKE.CORTEX.SUMMARIZE($${selected_transcript}$$) AS SUMMARY
"""
summary_result = session.sql(summary_query).collect()
summary = summary_result[0]['SUMMARY']
print(summary)


In [ ]:
# Define your categories
categories = ["Billing", "Technical Support", "Account Access", "Product Feedback"]

# Convert the list into a SQL array
category_list = ",".join([f"'{cat}'" for cat in categories])
sql_array = f"ARRAY_CONSTRUCT({category_list})"

# Classification query
classification_query = f"""
    SELECT 
        SNOWFLAKE.CORTEX.CLASSIFY_TEXT($${selected_transcript}$$, {sql_array}) AS CATEGORY
"""
classification_result = session.sql(classification_query).collect()
category = classification_result[0]['CATEGORY']
print(category)